In [1]:
#!/usr/bin/env python

# Copyright (c) 2018, ETH Zurich and UNC Chapel Hill.
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
#     * Redistributions of source code must retain the above copyright
#       notice, this list of conditions and the following disclaimer.
#
#     * Redistributions in binary form must reproduce the above copyright
#       notice, this list of conditions and the following disclaimer in the
#       documentation and/or other materials provided with the distribution.
#
#     * Neither the name of ETH Zurich and UNC Chapel Hill nor the names of
#       its contributors may be used to endorse or promote products derived
#       from this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE
# ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDERS OR CONTRIBUTORS BE
# LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR
# CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF
# SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS
# INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN
# CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE)
# ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE
# POSSIBILITY OF SUCH DAMAGE.
#
# Author: Johannes L. Schoenberger (jsch-at-demuc-dot-de)

import argparse
import numpy as np
import os
import struct


def read_array(path):
    with open(path, "rb") as fid:
        width, height, channels = np.genfromtxt(fid, delimiter="&", max_rows=1,
                                                usecols=(0, 1, 2), dtype=int)
        fid.seek(0)
        num_delimiter = 0
        byte = fid.read(1)
        while True:
            if byte == b"&":
                num_delimiter += 1
                if num_delimiter >= 3:
                    break
            byte = fid.read(1)
        array = np.fromfile(fid, np.float32)
    array = array.reshape((width, height, channels), order="F")
    return np.transpose(array, (1, 0, 2)).squeeze()


def write_array(array, path):
    """
    see: src/mvs/mat.h
        void Mat<T>::Write(const std::string& path)
    """
    assert array.dtype == np.float32
    if len(array.shape) == 2:
        height, width = array.shape
        channels = 1
    elif len(array.shape) == 3:
        height, width, channels = array.shape
    else:
        assert False

    with open(path, "w") as fid:
        fid.write(str(width) + "&" + str(height) + "&" + str(channels) + "&")

    with open(path, "ab") as fid:
        if len(array.shape) == 2:
            array_trans = np.transpose(array, (1, 0))
        elif len(array.shape) == 3:
            array_trans = np.transpose(array, (1, 0, 2))
        else:
            assert False
        data_1d = array_trans.reshape(-1, order="F")
        data_list = data_1d.tolist()
        endian_character = "<"
        format_char_sequence = "".join(["f"] * len(data_list))
        byte_data = struct.pack(endian_character + format_char_sequence, *data_list)
        fid.write(byte_data)


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("-d", "--depth_map",
                        help="path to depth map", type=str, required=True)
    parser.add_argument("-n", "--normal_map",
                        help="path to normal map", type=str, required=True)
    parser.add_argument("--min_depth_percentile",
                        help="minimum visualization depth percentile",
                        type=float, default=5)
    parser.add_argument("--max_depth_percentile",
                        help="maximum visualization depth percentile",
                        type=float, default=95)
    args = parser.parse_args()
    return args


In [2]:
import numpy as np
import os

def save_np(data, save_path):
    '''
    Input:
    ----------
    data: numpy.array
    save_path: str
        example: path/to/depth/image_name.npy
    '''
    ext = save_path.split('.')[-1].lower()
    def save_func(data, path):
        if ext == 'npy':
            save = np.save
        elif ext == 'npz':
            save = np.savez
        else:
            raise NotImplementedError(ext)
        with open(path, 'wb') as h:
            save(h, data)
    save_func(data, save_path)

In [3]:
import cv2
import pylab as plt

In [7]:
import os
path = '/home/cv_stu_3190102787/21_cv_final_project/MMW_BLD/dense/stereo/depth_maps/'  
f = open("test_list.txt")                
name = f.readline().strip("\n")     
m=0
while name:   
    img = path + name + ".geometric.bin"   
    depth = read_array( img )
    #mean = np.mean(depth)
    summ = 0
    num = 0
    ary = np.zeros([depth.shape[0]+2,depth.shape[1]+2] )
    for i in range(depth.shape[0]):
        for j in range(depth.shape[1]):
            ary[i+1][j+1] = depth[i][j]
            if depth[i][j] > 0:
                summ += depth[i][j]
                num += 1
    mean = summ / num

    for i in range(1, depth.shape[0]+1):
        for j in range( 1, depth.shape[1]+1):
            if( ary[i][j] ==0 ):
                tmp = 0
                cnt = 0
                for ii in range(3):
                    for jj in range(3):
                        if( ary[i-1+ii][j-1+jj] > 0 ):
                            tmp += ary[i-1+ii][j-1+jj]
                            cnt += 1
                if tmp == 0:
                    ary[i][j] = mean
                    depth[i-1][j-1] = mean
                else:
                    ary[i][j] = tmp/cnt
                    depth[i-1][j-1] = tmp/cnt
    save_path = "dpt/"+name[0:23]+".npz"
    data = cv2.bilateralFilter(depth,11,75,75)
    save_np(data, save_path)            
    name = f.readline().strip("\n")    
    print(m)
    m+=1
f.close()

0
1
2
3
4
5
6
7
8
9


In [6]:
import os
path = '/home/cv_stu_3190102787/21_cv_final_project/MMW_BLD/dense/stereo/depth_maps/'  
f = open("test_list.txt")                
name = f.readline().strip("\n")     
m=0
while name:   
    img = path + name + ".geometric.bin"   
    depth = read_array( img )
    #mean = np.mean(depth)
    ary = np.zeros([depth.shape[0]+2,depth.shape[1]+2] )
    
    for i in range(depth.shape[0]):
        for j in range(depth.shape[1]):
            if( depth[i][j] == 0 ):
                depth[i][j] = 9999
    min_d = np.min(depth)
    for i in range(depth.shape[0]):
        for j in range(depth.shape[1]):
            if( depth[i][j] == 9999 ):
                depth[i][j] = 0
                
    for i in range(depth.shape[0]):
        for j in range(depth.shape[1]):
            ary[i+2][j+2] = depth[i][j]
    for i in range(1, depth.shape[0]+1):
        for j in range( 1, depth.shape[1]+1):
            if( ary[i][j] == 0 ):
                tmp = 0
                cnt = 0
                for ii in range(3):
                    for jj in range(3):
                        if( ary[i-1+ii][j-1+jj] > 0 ):
                            tmp += ary[i-1+ii][j-1+jj]
                            cnt += 1
                if tmp == 0:
                    ary[i][j] = min_d * 1.62
                    depth[i-1][j-1] = min_d *1.62
                else:
                    ary[i][j] = tmp/cnt
                    depth[i-1][j-1] = tmp/cnt
    save_path = "dpt/"+name[0:23]+".npz"
    data = cv2.GaussianBlur(depth,(5,5),1.5)
    save_np(data, save_path)            
    name = f.readline().strip("\n")    
    print(m)
    m+=1
f.close()


0
1
2
3
4
5
6
7
8
9
